This notebook we will use to find and deploy the most affordable GPU's on the market with the Shadeform API. We limit our search to machines with 1x80GB A100 machines, but that is easily configurable below.

In [ ]:
import requests
import json


In [ ]:
base_url = "https://api.shadeform.ai/v1/instances"
instance_type_url = base_url + "/types"
create_url = base_url + "/create"
headers = {
    "X-API-KEY": "<Add-your-key-here>", 
    "Content-Type" : "application/json"
}
a100_shade_instance_type = "A100_80G"

In [ ]:
response = requests.request("GET", instance_type_url, headers=headers)

print(response.text)



In [ ]:
instance_types = json.loads(response.text)["instance_types"]
lowest_price = None
cheapest_instance = None
region = None

for instance in instance_types:
    if instance["shade_instance_type"] == a100_shade_instance_type:
        if lowest_price is None or lowest_price > instance["hourly_price"]:
            available = False
            for availabilities in instance["availability"]:
                if availabilities["available"]:
                    region = availabilities["region"]
                    available = True
                    break
            if available:
                lowest_price = instance["hourly_price"]
                cheapest_instance = instance

if cheapest_instance is not None:
    print(f"The cheapest {a100_shade_instance_type} instance is:", cheapest_instance)
else:
    print(f"No instances of type {a100_shade_instance_type} found.")

In [ ]:
payload = {
    "cloud" : cheapest_instance["cloud"],
    "name" : "cool_gpu_server",
    "region" : region,
    "shade_cloud" : True,
    "shade_instance_type" : a100_shade_instance_type
}

response = requests.request("POST", create_url, json=payload, headers=headers)


In [ ]:
print(response.text)